In [5]:
#import necessary packages and rename
import sys                   #system-specific parameters and functions
import numpy as np           #arrays, matrices, math functions library
import pandas as pd          #data manipulation and analysis library
import matplotlib.pyplot as plt     #plotting library
import seaborn as sns        #data visualization library
import scipy                 #scientific and technical computing library
import sklearn               #classification, regression, clustering algorithms library
from sklearn.metrics import classification_report, accuracy_score
from sklearn.covariance import EllipticEnvelope

#load the dataset from the csv file using pandas
data = pd.read_csv('creditcard.csv') 

data = data.sample(frac=0.1, random_state = 1)

#Determine number of fraud cases in dataset
Fraud = data[data['Class'] == 1]
Valid = data[data['Class'] == 0]

outlier_fraction = len(Fraud) / float(len(Valid))
#Get all the columns from the DataFrame
columns = data.columns.tolist()

#filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Class"]]

#store variable we will be predicting on
target = "Class"

X = data[columns]
Y = data[target]

state = 1

#Fit the model
n_outliers = len(Fraud)

#splitter = StratifiedShuffleSplit(n_splits = 1, test_size=0.5, random_state=0)
def normalize(X):
    for feature in X.columns:
        X[feature] -= X[feature].mean()
        X[feature] /= X[feature].std()
    return X

clf = EllipticEnvelope(contamination=outlier_fraction, support_fraction=1)
clf.fit(X)
y_pred = clf.predict(X)
scores_pred = clf.decision_function(X)

y_pred[y_pred==1] = 0
y_pred[y_pred==-1]= 1

n_errors = (y_pred != Y).sum()

#run classification metrics
print('{}:{}'.format("Elliptic Envelope", n_errors))
print(accuracy_score(Y, y_pred))
print(classification_report(Y, y_pred))

Elliptic Envelope:99
0.9965239984551104
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     28432
          1       0.00      0.00      0.00        49

avg / total       1.00      1.00      1.00     28481

